# Recreational center usage: how does it affect grades?

I was fortunate to get access to a large volume of data of student swipes at my university's recreational center. This is all swipes for a complete academic year, anonymized of course. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
import seaborn as sns
raw_data = pd.read_excel('Anonymized - 2017 Summer, 2017 Fall, 2018 Spring CoRec Swipe Data.xlsx',header=1)
raw_data.head()

,PUID,Gender,Admission Population Group,Residency,Ethnicity,International?,Geocluster,URM,Citizenship Type,Nation of Citizenship,...,Spring Swipes,Fall?,Fall Swipes,Summer?,Summer Swipes,Year Swipes,Semesters,Per semester Swipe,CoRec User,Type of User
0,1,F,New First Time,Foreign,International,international,South Asia,N,A,Indonesia,...,10,T,20,T,39,69,3,23.0,T,Moderate
1,2,M,New First Time,Foreign,International,international,East Asia,N,A,China,...,0,T,1,F,0,1,2,0.5,T,Infrequent
2,3,M,New First Time,Foreign,International,international,South Asia,N,A,India,...,27,T,24,F,0,51,2,25.5,T,Moderate
3,4,M,New First Time,Non-Resident,White,domestic,USA,N,C,NaN,...,0,T,0,T,0,0,3,0.0,F,Zero use
4,5,F,New First Time,Resident,Black or African American,domestic,USA,Y,C,NaN,...,14,T,22,F,1,37,2,18.5,T,Moderate


In [2]:
data = raw_data
data = data[data['Overall GPA'].notna()]
data

,PUID,Gender,Admission Population Group,Residency,Ethnicity,International?,Geocluster,URM,Citizenship Type,Nation of Citizenship,...,Spring Swipes,Fall?,Fall Swipes,Summer?,Summer Swipes,Year Swipes,Semesters,Per semester Swipe,CoRec User,Type of User
0,1,F,New First Time,Foreign,International,international,South Asia,N,A,Indonesia,...,10,T,20,T,39,69,3,23.0,T,Moderate
1,2,M,New First Time,Foreign,International,international,East Asia,N,A,China,...,0,T,1,F,0,1,2,0.5,T,Infrequent
2,3,M,New First Time,Foreign,International,international,South Asia,N,A,India,...,27,T,24,F,0,51,2,25.5,T,Moderate
3,4,M,New First Time,Non-Resident,White,domestic,USA,N,C,NaN,...,0,T,0,T,0,0,3,0.0,F,Zero use
4,5,F,New First Time,Resident,Black or African American,domestic,USA,Y,C,NaN,...,14,T,22,F,1,37,2,18.5,T,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39250,39251,F,Transfer,Resident,Asian,Domestic,USA,N,P,NaN,...,0,F,0,T,33,33,1,33.0,T,Heavy
39251,39252,F,New First Time,Resident,Asian,Domestic,USA,N,C,NaN,...,0,F,0,T,42,42,1,42.0,T,Heavy
39252,39253,F,New First Time,Foreign,International,International,East Asia,N,A,China,...,0,F,0,T,0,0,1,0.0,F,Zero use
39253,39254,F,New First Time,Foreign,International,International,NaN,N,A,Pakistan,...,0,F,0,T,18,18,1,18.0,T,Moderate


### Exploratory data analysis and data cleaning

In [3]:
from pandas_profiling import ProfileReport

ModuleNotFoundError: No module named 'pandas_profiling'

In [ ]:
profile = ProfileReport(data)

In [ ]:
profile


In [ ]:
data.columns

Yikes, that's 60 columns. But there are a lot of columns that are highly collinear and some others that are missing a lot of values. We will remove these as they contain no information. We will use the results from our earlier profiling to select the columns to delete. We will remove all columns that have >50% missing values, which is not a totally unreasonable thing to do. It also happens that it is intuitively unlikely for these columns to have any effect on the outcome. 

In [ ]:
columns_to_drop = ['PUID','Major 2','Major 3','Major 4','Minor 1','Minor 2','Minor 3','Minor 4',
                   '1st Concentration','Overall Credits Attempted','Nation of Citizenship','Residence Hall']
data.drop(columns_to_drop, axis=1, inplace=True)
data = data.drop(data[data['CoRec User'] == 'F'].index)


In [ ]:
# subset1 = data.loc[data['CoRec User'] == 'T']
# subset1 = data.groupby('Type of User', as_index=False)['Overall GPA'].mean()
%pylab inline
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.boxplot(x="Type of User", y="Overall GPA", data=data,
                 showfliers=False, order=["Infrequent", "Moderate", "Heavy"])
sns.set_context("paper", rc={"font.size":40,"axes.titlesize":40,"axes.labelsize":40})   
plt.show()

This is a fantastic early indication that the frequency of use is strongly correlated with GPA. 

In [ ]:
data['Type of User'].value_counts()

In [ ]:
bins = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4]
labels = ['A','B','C','D','E','F','G','H']
data['binnedgrade'] = pd.cut(data['Overall GPA'], bins=bins, labels=labels).astype(str)

swipe_cols = [col for col in data.columns if 'GPA' in col]

data.drop(swipe_cols, axis=1, inplace=True)
data

In [ ]:
cat = len(data.select_dtypes(include=['object']).columns)
num = len(data.select_dtypes(include=['int64','float64']).columns)
print('Total Features: ', cat, 'categorical', '+',
      num, 'numerical', '=', cat+num, 'features')

In [ ]:
data.isnull().sum()

We have to be careful about how we handle the missing values - for instance, NaN for residence hall could just mean that the student did not use on-campus housing. So, NaN in that case could actually provide useful information. From the above breakdown, this is what we will plan to do with the NaN values:
1. Geocluster: Impute with mode
2. Citizenship type: Leave as is- i.e., forms a unique value of its own
3. Academic School Grouping, Program: Impute with mode
4. Year GPA: Impute with median
5. Semester Honors: Leave as is
6. Spring Credits Attempted, Spring Credits Earned, Spring GPA: impute with median
All this magic happens below:

In [ ]:
data['Geocluster'].fillna(data['Geocluster'].mode().iloc[0],inplace=True)
data['Academic School Grouping'].fillna(data['Academic School Grouping'].mode().iloc[0],inplace=True)
data['Program'].fillna(data['Program'].mode().iloc[0],inplace=True)
# data['Year GPA'].fillna(data['Year GPA'].median(),inplace=True)
data['Spring Credits Attempted'].fillna(data['Spring Credits Attempted'].median(),inplace=True)
data['Spring Credits Earned'].fillna(data['Spring Credits Earned'].median(),inplace=True)
# data['Spring GPA'].fillna(data['Spring GPA'].median(),inplace=True)

Now we are on to encoding the features. The categorical features are encoded using OneHotEncoder, the output is encoded using Label encoder. 

In [ ]:
y = data.iloc[:,-1]
le = LabelEncoder()
y = le.fit_transform(y)
y = pd.DataFrame(y,columns=['binnedgrade'])

X = data.iloc[:,:-1]
X = pd.get_dummies(X, prefix_sep='_')

Now we will apply standard scaler to the features. 

In [ ]:
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=44)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [ ]:
y

Looks great. Let's train and test then. 

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
y_pred_dummy = dummy_clf.predict(X_train)
print('Dummy: Accuracy=%.3f' % (accuracy_score(y_train, y_pred_dummy)))
print (confusion_matrix(y_train, y_pred_dummy))


In [ ]:
# Training the data and predicting 
# Logistic regression
from sklearn.linear_model import LogisticRegression
# fit a model
lrclf = LogisticRegression(solver='lbfgs',max_iter = 1000)
lrclf.fit(X_train, y_train)
y_pred_lr = lrclf.predict(X_test)
print('Logistic: Accuracy=%.3f' % (accuracy_score(y_test, y_pred_lr)))

In [ ]:
# Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
rfclf = RandomForestClassifier(max_depth=50, random_state=0,n_estimators=200)
rfclf.fit(X_train, y_train)
y_pred_rf = rfclf.predict(X_test)
print('Random Forest: Accuracy=%.3f' % (accuracy_score(y_test, y_pred_rf)))
print (confusion_matrix(y_test, y_pred_rf))


Let's see if we can improve the accuracy using an Adaboost classfier. 

In [ ]:
# SVM classifier
from sklearn import svm
svmclf = svm.SVC(kernel='rbf')
svmclf.fit(X_train, y_train)
y_pred_svm = svmclf.predict(X_test)
print('SVM: Accuracy=%.3f' % (accuracy_score(y_test, y_pred_svm)))
print (confusion_matrix(y_test, y_pred_svm))


In [ ]:
X_train2 = X_train.iloc[:,:10]
X_test2 = X_test.iloc[:,:10]


In [ ]:
# Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
rfclf = RandomForestClassifier(max_depth=50, random_state=0,n_estimators=100)
rfclf.fit(X_train, y_train)
y_pred_rf = rfclf.predict(X_test)
print('Random Forest: Accuracy=%.3f' % (accuracy_score(y_test, y_pred_rf)))
print (confusion_matrix(y_test, y_pred_rf))